# IMDB Scraping tool v2
This is a scraping script that gets data for every episode of a tv show from the website IMDB.com. It consists of two scraping functions that get relevant data and produces a dataframe and a excel file ready to be analyzed.
It also uses multiprocessing library to make use of multiple threads. Thanks to that parsing is now faster than before.

In [1]:
%%time
import multiprocessing  
import random
from multiprocessing import Pool
from scrapers import *
idnlist=['tt0903747',
'tt1632701',
'tt4158110',
'tt2575988',
'tt3322314',
'tt3322312',
'tt2357547',
'tt3322310',
'tt5675620',
'tt4230076',
'tt3032476',
'tt2442560',
'tt1837492',
'tt5420376',
'tt4270492',
'tt1475582',
'tt4800878',
'tt2356777',
'tt2707408',
'tt2802850',
'tt0475784',
'tt4574334',
'tt5753856',
'tt2085059',
'tt2261227',
'tt5071412',
'tt5290382',
'tt5028002',
'tt4118466',
'tt0290978',
'tt4422836',
'tt5296406',
'tt6468322',
'tt0386676',
'tt6586318',
'tt7660850',
'tt7134908',
'tt6258718',
'tt1266020',
'tt8714904',
'tt5788792',
'tt4922804',
'tt1870479',
'tt1312171',
'tt7889220',]
# Pool tells how many at a time
with Pool(16) as p:
    dfs1 = p.map(parse_and_get_df, idnlist)
    dfs2 = p.map(parse_and_get_df2, idnlist)

#create first df
if len(dfs1)>1:
    print(len(dfs1))
    df4=pd.concat(dfs1)
else:
    df4=pd.concat(dfs1)
#create 2nd df
if len(dfs2)>1:
    print(len(dfs2))
    df5=pd.concat(dfs2)
else:
    df5=pd.concat(dfs2)
#tidy genre column
genre_df = df5['genre'].apply(lambda x: pd.Series(x.split(',')))
genre_df.rename(columns={0:'Genre 1',1:'Genre 2',2:'Genre3'},inplace=True)
df6=pd.concat([df5,genre_df],axis=1)
df6['len']=df6['len'].str.extract('(\d+)', expand=True)
df6['votes']=df6['votes'].str.replace(',', '')
df6=df6.drop('genre',1)
df6

#merge both dfs, using dedup technique, counting duplicates
a = df4.assign(D=df4.groupby(["series", "name"]).cumcount())
b = df6.assign(D=df6.groupby(["series", "name"]).cumcount())

dffinal2=a.merge(b, on=["series", "name", 'D'], how='left').drop('D', 1)

writer = pd.ExcelWriter('imdx.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
dffinal2.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

45
45
Wall time: 35.5 s


In [3]:
dffinal2

,series,name,season,number,rating,len,year,votes,Genre 1,Genre 2,Genre3
0,Breaking Bad,Pilot,1,1.0,8.9,58,2008,20110,Crime,Drama,Thriller
1,Breaking Bad,Cat's in the Bag...,1,2.0,8.7,48,2008,14198,Crime,Drama,Thriller
2,Breaking Bad,...And the Bag's in the River,1,3.0,8.7,48,2008,13765,Crime,Drama,Thriller
3,Breaking Bad,Cancer Man,1,4.0,8.2,48,2008,13402,Crime,Drama,Thriller
4,Breaking Bad,Gray Matter,1,5.0,8.3,48,2008,13103,Crime,Drama,Thriller
5,Breaking Bad,Crazy Handful of Nothin',1,6.0,9.2,48,2008,15118,Crime,Drama,Thriller
6,Breaking Bad,A No-Rough-Stuff-Type Deal,1,7.0,8.8,48,2008,13326,Crime,Drama,Thriller
7,Breaking Bad,Seven Thirty-Seven,2,1.0,8.7,47,2009,11866,Crime,Drama,Thriller
8,Breaking Bad,Grilled,2,2.0,9.2,46,2009,13955,Crime,Drama,Thriller
9,Breaking Bad,Bit by a Dead Bee,2,3.0,8.3,47,2009,11559,Crime,Drama,Thriller
